In [189]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
#from collections import Counter
#import nltk

In [190]:
bibleraw = pd.read_json('https://raw.githubusercontent.com/bibleapi/bibleapi-bibles-json/master/asv.json', lines = True)

In [191]:
bibleraw.head()

,book_id,book_name,chapter,text,translation_id,verse
0,Gen,Genesis,1,In the beginning God created the heavens and t...,ASV,1
1,Gen,Genesis,1,And the earth was waste and void; and darkness...,ASV,2
2,Gen,Genesis,1,"And God said, Let there be light: and there wa...",ASV,3
3,Gen,Genesis,1,"And God saw the light, that it was good: and G...",ASV,4
4,Gen,Genesis,1,"And God called the light Day, and the darkness...",ASV,5


In [32]:
#accidentally download the quran in original arabic, unfortunately it can't be useful.
quranraw = open('../__DATA__/Holy Texts/quran-simple.txt','r')
quranraw = quranraw.read()
quranraw[:1000]

'بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ\nالْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ\nالرَّحْمَنِ الرَّحِيمِ\nمَالِكِ يَوْمِ الدِّينِ\nإِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ\nاهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ\nصِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ الْمَغْضُوبِ عَلَيْهِمْ وَلَا الضَّالِّينَ\nبِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ الم\nذَلِكَ الْكِتَابُ لَا رَيْبَ فِيهِ هُدًى لِلْمُتَّقِينَ\nالَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ الصَّلَاةَ وَمِمَّا رَزَقْنَاهُمْ يُنْفِقُونَ\nوَالَّذِينَ يُؤْمِنُونَ بِمَا أُنْزِلَ إِلَيْكَ وَمَا أُنْزِلَ مِنْ قَبْلِكَ وَبِالْآخِرَةِ هُمْ يُوقِنُونَ\nأُولَئِكَ عَلَى هُدًى مِنْ رَبِّهِمْ وَأُولَئِكَ هُمُ الْمُفْلِحُونَ\nإِنَّ الَّذِينَ كَفَرُوا سَوَاءٌ عَلَيْهِمْ أَأَنْذَرْتَهُمْ أَمْ لَمْ تُنْذِرْهُمْ لَا يُؤْمِنُونَ\nخَتَمَ اللَّهُ عَلَى قُلُوبِهِمْ وَعَلَى سَمْعِهِمْ وَعَلَى أَبْصَارِهِمْ غِشَاوَةٌ وَلَهُمْ عَذَابٌ عَظِيمٌ\nوَمِنَ النَّاسِ مَنْ يَقُولُ آمَنَّا بِاللَّهِ وَبِالْيَوْمِ الْآخِرِ وَمَا هُمْ بِمُؤْمِنِينَ\nيُخَادِعُونَ اللَّهَ وَ

In [49]:
import glob
import os

file_list = glob.glob(os.path.join(os.getcwd(), "../__DATA__/Holy Texts/quran-verse-by-verse-text", "*.txt"))
file_list.sort()

quran_verses = []

for file_path in file_list:
    with open(file_path) as f_input:
        quran_verses.append(f_input.read())

Ways of organizing the analysis:
- We will be performing a model that classifies OT vs. NT. 

In [50]:
len(quran_verses)

6349

In [67]:
quran_verses[6348]

"The Quran\nModern English Translation\nVerse by Verse\n\nTranslated by Talal Itani\nwww.ClearQuran.com\n\nAvailable in two editions. This edition (A) uses the word 'Allah' to refer to the Creator. Edition (B) uses the word 'God'.\n\nThese files can be shared and distributed\nProvided under the Creative Commons License\nAttribution-NonCommercial-NoDerivs\n\n"

In [69]:
#but it doesn't matter anyways because I just decided that I don't to use the Quran.

In [192]:
bibleraw['book_name'].unique()

array(['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy',
       'Joshua', 'Judges', 'Ruth', '1 Samuel', '2 Samuel', '1 Kings',
       '2 Kings', '1 Chronicles', '2 Chronicles', 'Ezra', 'Nehemiah',
       'Esther', 'Job', 'Psalms', 'Proverbs', 'Ecclesiastes',
       'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel',
       'Daniel', 'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah',
       'Nahum', 'Habakkuk', 'Zephaniah', 'Haggai', 'Zechariah', 'Malachi',
       'Matthew', 'Mark', 'Luke', 'John', 'Acts of the Apostles',
       'Romans', '1 Corinthians', '2 Corinthians', 'Galatians',
       'Ephesians', 'Philippians', 'Colossians', '1 Thessalonians',
       '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
       'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John',
       '3 John', 'Jude', 'Revelation'], dtype=object)

In [193]:
#Creating list of Old Testament and New Testament Chapters
OT = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy',
       'Joshua', 'Judges', 'Ruth', '1 Samuel', '2 Samuel', '1 Kings',
       '2 Kings', '1 Chronicles', '2 Chronicles', 'Ezra', 'Nehemiah',
       'Esther', 'Job', 'Psalms', 'Proverbs', 'Ecclesiastes',
       'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel',
       'Daniel', 'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah',
       'Nahum', 'Habakkuk', 'Zephaniah', 'Haggai', 'Zechariah', 'Malachi']
NT = ['Matthew', 'Mark', 'Luke', 'John', 'Acts of the Apostles',
       'Romans', '1 Corinthians', '2 Corinthians', 'Galatians',
       'Ephesians', 'Philippians', 'Colossians', '1 Thessalonians',
       '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
       'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John',
       '3 John', 'Jude', 'Revelation']

In [194]:
bible = bibleraw.copy()
bible['testament'] = ''
bible['book_chapter'] = ''
bible.loc[bible['book_name'].isin(OT), 'testament'] = 'OT'
bible.loc[bible['book_name'].isin(NT), 'testament'] = 'NT'

In [195]:
#Creating a new dataframe with bible with each row as a chapter. These rows will be our documents.
bible_chapters = pd.DataFrame(columns = bible.columns)
i = 0
books = bible.book_name.unique()
for book in books:
    chapters = bible[bible['book_name'] == book].chapter.unique()
    for chapter in chapters:
        book_chapter = book + ' ' + str(chapter)
        text = bible[(bible['book_name'] == book) & (bible['chapter'] == chapter)].text
        if book in OT:
            testament = 'OT'
        elif book in NT:
            testament = 'NT'
        bible_chapters.loc[i, ['book_name','chapter','book_chapter','text','testament']] = [book, chapter, book_chapter, ' '.join(text), testament]
        i += 1

In [196]:
bible_chapters = bible_chapters[['book_name','chapter','book_chapter','text','testament']]
bible_chapters.head()

,book_name,chapter,book_chapter,text,testament
0,Genesis,1,Genesis 1,In the beginning God created the heavens and t...,OT
1,Genesis,2,Genesis 2,"And the heavens and the earth were finished, a...",OT
2,Genesis,3,Genesis 3,Now the serpent was more subtle than any beast...,OT
3,Genesis,4,Genesis 4,And the man knew Eve his wife; and she conceiv...,OT
4,Genesis,5,Genesis 5,This is the book of the generations of Adam. I...,OT


In [207]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

# Utility Functions for word tokenizing document for BOW analysis
def bag_of_bags(document_list):
    
    '''Creating bags of differents spacy token types. Outputs 5 different
    bags. Utilize the function as follows:
    
    allwords, allents, allnps, alltags, alltokens = bag_of_bags(document_list)
    '''
    
    #Intializing Text Parser
    from collections import Counter
    nlp = spacy.load('en')

    #Parsing the Text
    text = ' '.join(document_list)
    text = nlp(text)
        
    # Words
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    allwords = [item[0] for item in Counter(allwords).most_common(1000)]
    
    # Entities
    allents = [str(ent)+' ENT' for ent in text.ents]
    allents = [item[0] for item in Counter(allents).most_common(250)]

    # Noun Phrases
    allnps = [str(np)+' NPHR' for np in text.noun_chunks]
    allnps = [item[0] for item in Counter(allnps).most_common(500)]
    
    # Words with POS Tags
    alltags = [token.lemma_+' '+token.tag_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    alltags = [item[0] for item in Counter(alltags).most_common(1000)]
    
    # combining all token categories into one list
    alltokens = allwords + allents + allnps + alltags
    
    return allwords, allents, allnps, alltags, alltokens
    
def bob_features(document_list):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=alltokens)
    df['document_text'] = document_list
    df.loc[:, alltokens] = 0
    
    # Process each row, counting the occurrence of tokens in each sentence.
    for i, document in enumerate(df['document_text']):
        
        document = nlp(document)
        
        words = [token.lemma_
                 for token in document
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in allwords
                 )]
        
        ents = [str(ent)+' ENT'
                 for ent in document.ents
                 if str(ent) + ' ' in allents
                 ]
        
        nps = [str(np)+' NPHR'
                 for np in document.noun_chunks
                 if str(np)+' NPHR' in allnps
                 ]
        
        tags = [token.lemma_+' '+token.tag_
                 for token in document
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and (token.lemma_+' '+token.tag_) in alltags
                 )]
        
        tokens = words + ents + nps + tags
        
        # Populate the row with word counts.
        for token in tokens:
            df.loc[i, token] += 1
            
        # This counter is just to make sure the kernel didn't hang.
        if i % 10 == 0:
            print("Processing row {}".format(i))

    return df

# Utility function for word tokenizing documents for tf-idf preparation
def document_tokenizer(raw_documents_list):
     
    '''Takes a list of documents and returns corresponding list (same shape) of lemmatized tokens,
     entities, noun_phrases, and POS_tagged words all returned as strings. Each token has spaces replaced
     with underscores _ so that tfidfvectorizor can process each token as a single word.
     
     raw_documents_list should be cleaned of any text that spacy can't process.'''

    #Initialize Output List of Document Tokens, and SpaCy NLP Preprocessor
    tokenized_documents = []
    
    #individually spacy process the documents in the original documents list
    documents = [ nlp(document) for document in raw_documents_list ]
    
    #tokenizing text by token category
    for text in documents:
    
        allwords = [token.lemma_
                    for token in text
                    if not token.is_punct
                    and not token.is_stop]

        allents = []
        for ent in text.ents:
            string_ent = str(ent).strip('[]').strip().replace(' ','_') #Spaces in token replaced with underscore #also remove extraneous brackets as last second tweak
            allents.append(string_ent + '_ENTITY') #label as entity, differentiate from words

        allnps = []
        for np in text.noun_chunks:
            string_np = str(np).strip('[]').strip().replace(' ','_') #Spaces in token replaced with underscore #also remove extraneous brackets as last second tweak
            allnps.append(string_np + '_NPHRASE') #label as NPHRASE, differentiate from words

        alltags = [token.lemma_ + '_' + token.tag_ #Spaces in token replaced with underscore
                    for token in text
                    if not token.is_punct
                    and not token.is_stop]
        
        #aggregating all document token categories into one string per document and appending list of strings
        #to the final tokenized document list
        doc_tok_cats = [allwords, allents, allnps, alltags] 
        doc_tokens = [' '.join(cat) for cat in doc_tok_cats]
        tokenized_documents.append(' '.join(doc_tokens)) 
        
    return tokenized_documents

In [140]:
chapters_clean = []
for chapter in bible_chapters.text:
    chapters_clean.append(text_cleaner(chapter))

In [232]:
#checking the length.
len(' '.join(chapters_clean))

4117953

## Bag of Words (Bag of bags) Features and Modeling to predict which testament a chapter is from.

In [180]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Splitting the data into training and test sets. The sizes of each set can not exceed
# 1 million characters in text due to computing limitations, although this number can be changed.
bible_df = bible_chapters.copy()
bible_df['text'] = chapters_clean
train_set, test_set = train_test_split(bible_df, train_size = 0.2, random_state = 5)
test_set = resample(test_set, n_samples = len(text_train), random_state = 10)

In [208]:
# Stitching the bags together; creating these bags is similar to fitting the model,
# since it requires feature extraction of a training set.
allwords, allents, allnps, alltags, alltokens = bag_of_bags(train_set.text)


In [209]:
train_counts = bob_features(list(train_set.text))

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230


In [210]:
test_counts = bob_features(list(test_set.text))

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230


In [211]:
#adding 
train_counts['testament'] = train_set['testament']

In [231]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

X_train = train_counts.loc[:, ~(train_counts.columns).isin(['document_text','testament'])]
Y_train = train_set['testament']

clf = LogisticRegression()
clf.fit(X_train,Y_train)
predict = clf.predict(test_counts.loc[:, ~(test_counts.columns).isin(['document_text'])])
conmat = confusion_matrix(test_set['testament'], predict)
display(conmat)
clf.score(test_counts.loc[:, ~(test_counts.columns).isin(['document_text'])], test_set['testament'])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[ 42,   1],
       [  2, 192]])

0.9873417721518988

The model turned out great. Prediction accuracy when rounded is 99%

# Predictive Model using TF-IDF Features

In [234]:
# Creating column to tokenize each chapter 
nlp = spacy.load('en')
bible_df['token_text'] = document_tokenizer(bible_df.text)

In [245]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating training and test sets for TF-IDF
train_set, test_set = train_test_split(bible_df, train_size = 0.2, random_state = 6)
#test_set = resample(test_set, n_samples = len(text_train), random_state = 11)

X_train = train_set['token_text']
Y_train = train_set['testament']
X_test = test_set['token_text']
Y_test = test_set['testament']

#TF-IDF Transforming the text using X_train training set
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=False, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
print("Number of features: %d" % X_train_tfidf.get_shape()[1])


Number of features: 10822


In [246]:
# Creating a predictive model
clf = LogisticRegression()
clf.fit(X_train_tfidf, Y_train)
predict = clf.predict(X_test_tfidf)
conmat = confusion_matrix(Y_test, predict)
display(conmat)
clf.score(X_test_tfidf, Y_test)

array([[136,  60],
       [  0, 756]])

0.9369747899159664

In [244]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=False, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )
clf = LogisticRegression()

tfidf_clf_model = Pipeline(steps=[('tfidf', vectorizer),('lreg classifier', clf)])
cross_val_score(tfidf_clf_model, bible_df['token_text'], bible_df['testament'], cv = 5)



/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.94957983, 0.98319328, 0.94957983, 0.97478992, 0.88185654])

The cross validation scores all look great besides the last fold, which creates a 10% spread between itself and the highest score